Setup code

In [99]:
%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY in .env file
# UPSTAGE_API_KEY=your_api_key

import warnings
warnings.filterwarnings("ignore")

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [100]:
text = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
demonstrating superior performance in various natural language processing (NLP) tasks. 
Inspired by recent efforts to efficiently up-scale LLMs, 
we present a method for scaling LLMs called depth up-scaling (DUS), 
which encompasses depthwise scaling and continued pretraining.
In contrast to other LLM up-scaling methods that use mixture-of-experts, 
DUS does not require complex changes to train and inference efficiently. 
We show experimentally that DUS is simple yet effective 
in scaling up high-performance LLMs from small ones. 
Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
a variant fine-tuned for instruction-following capabilities, 
surpassing Mixtral-8x7B-Instruct. 
SOLAR 10.7B is publicly available under the Apache 2.0 license, 
promoting broad access and application in the LLM field.
"""

query = f"""
Make 3 questions using this text.

{text}
"""

query2 = """
Linear transformation is a function that satisfies two features.
One is homogeneity, and other is additivity.

Linear transformation of the subspace is also subspace.
Prove or disprove this statement based on given contents.
"""

query3 = """
The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
"""

query4 = """
What is a Mahjong game?
"""

query5 = """
Minimize the cost of the boolean function f(x, y, z) = m(0) + m(1) + m(2).
m(n) means minterm of the binary number n.
"""

In [101]:
from langchain_upstage import ChatUpstage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatUpstage()

message = query5

prompt_template = PromptTemplate.from_template(
    """
You are the AI that separates instruction and context from the query.
In this query, separate the instruction and context.
Do not answer to the query, just separate it. Also, only use given query itself, do not generate or change any words.
Instruction is saying about what AI should do.
Context is can be a base information or some text to convert or manipulate.
Context can be not provided, then print context to space.
Instruction and context must not have duplicate contents.
Print instruction as first paragraph and context as second paragraph.
---
Query : {query}
"""
)
chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"query": message})

instr, context = [s.strip() for s in answer.split("Instruction:")[1].split("Context:")]
print(instr)
print(context)

Minimize the cost of the boolean function f(x, y, z) = m(0) + m(1) + m(2).
m(n) means minterm of the binary number n. For example, m(1) = x'y'z.


In [102]:
import os
from tavily import TavilyClient

tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

scontext = str(tavily.search(query=instr))

prompt_template = PromptTemplate.from_template(
    """
For instruction "{instr}", divide instruction into few steps.
Do not answer or calculate for each steps, and only divide instructions to the smaller problem.
Only print 1., 2., 3., ... steps to do. Each step should be a one-liner.
Refer to the context if you need : {context}
Searched context to help dividing instructions : {scontext}
"""
)

chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"instr": instr, "context": context, "scontext": scontext}).split("\n")
print(answer)
print(len(answer))

['1. Understand the concept of K-Map and its application in simplifying boolean functions.', '2. Identify the prime implicants and essential prime implicants for the given boolean function f(x, y, z) = m(0) + m(1) + m(2).', '3. Apply the selection rule to find the minimum set of essential prime implicants that can be used to represent the function.', '4. Verify the simplified function using a truth table or other methods.']
4


In [103]:
prompt_template = PromptTemplate.from_template(
    """
For instruction "{instr}", determine instruction can be done only using context.
Just answer it in "Yes" or "No".
---
Context : {context}
"""
)

chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"instr": instr, "context": context})
print(answer)

add = ""

if(answer == "No"): # find context from tavily
    context = scontext
    add = "\nAnswer with references."

Yes


In [104]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

rag_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
    ]
)

chain = rag_with_history_prompt | llm | StrOutputParser()

history = []

for i in range(len(answer)):
    if (i == 0 and len(context) != 0):
        history.append(
            HumanMessage(answer[i] + "\n Refer to or use this context.\n Context:" + context)
        )
    else:
        history.append(
            HumanMessage(answer[i]) 
        )
    chain_result = chain.invoke({"history": history})
    history.append(
        AIMessage(chain_result)
    )

history.append(
    HumanMessage(message + add)
)
answer = chain_result = chain.invoke({"history": history})
print(answer)


To minimize the cost of the boolean function f(x, y, z) = m(0) + m(1) + m(2), we need to find the simplest expression for this function using the given minterms.

First, let's find the minterms m(0), m(1), and m(2):

- m(0) = x'y'z' (since 0 in binary is 000, all variables are complemented)
- m(1) = x'y'z (since 1 in binary is 001, only z is not complemented)
- m(2) = x'yz' (since 2 in binary is 010, only y and z are not complemented)

Now, let's add these minterms together:

f(x, y, z) = m(0) + m(1) + m(2)
f(x, y, z) = x'y'z' + x'y'z + x'yz'

We can simplify this expression by combining the terms with the same variables:

f(x, y, z) = x'y'z' + x'yz'
f(x, y, z) = x'y'z' + x'yz'

Since both terms have x'y' and x'y as common factors, we can factor them out:

f(x, y, z) = x'(y'(z' + z))

Now, z' + z is always true because either z is true and z' is false, or z is false and z' is true. So, the expression simplifies to:

f(x, y, z) = x'

This means that the cost of the boolean function f(x,